
# Extração de Tópicos com Non-negative Matrix Factorization (NMF)


Exemplos oferecidos pelo próprio scikit-learn no seguinte link:

* https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py




# Novo QGS

Iremos adotar a partir de agora um novo QGS, com 52 artigos relacionados a mineração de texto em auxílio à revisões sistemáticas da literatura, encontrados [aqui](Files-QGS/QGS-ia-pdf). Também está disponível uma planilha [aqui](Files-QGS/QGS.ods) com todos os nomes destes arquivos e se estes estão localizados na base de dados Scopus ou não. A principio, utilizaremos um dataset do QGS em formato .txt que possui apenas o título, palavras-chave e o resumo destes artigos, localizado [aqui](Files-QGS/QGS-ia-txt/metadata/).

Importante salientar que dos 52 arquivos utilizados como QGS **46** estão presentes na base de dados Scopus (Ao se procurar pelo titulo destes, são encontrados com facilidade).

Abaixo segue um código exemplo de como foi acessado tal base e efetuado novamente os testes realizados com o QGS anterior, alterando apenas os valores de number_topics e number_words.

In [2]:
# encoding: utf-8
from __future__ import print_function
from time import time
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

number_topics = 5
number_words = 5
max_document_frequency = 1.0
min_document_frequency = 0.4
ngram = (1, 3)
max_features = None

# Imprime os tópicos com as palavras em ordem
def print_top_words(model, feature_names, number_words):
    for topic_index, topic in enumerate(model.components_):
        message = "Topic %d: " % (topic_index + 1)
        message += "{"
        message += " - ".join([feature_names[i]
                               for i in topic.argsort()[:-number_words - 1:-1]])
        message += "}\n"
        print(message)

# Carrega o dataset de treinamento
files = load_files(container_path = '/home/fuchs/Documentos/MESTRADO/Masters/Files-QGS/QGS-ia-txt/metadata', encoding="iso-8859-1")

# Usa tf-idf para o NMF.
tfidf_vectorizer = TfidfVectorizer(max_df = max_document_frequency,
                                   min_df = min_document_frequency,
                                   ngram_range= ngram,
                                   max_features = max_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(files.data)

# NMF 1
print("Rodando NMF (Frobenius norm) com tf-idf... \n")

# Salva os nomes das palavras em um dicionário
dic = tfidf_vectorizer.get_feature_names()

nmf = NMF(n_components = number_topics,
          random_state = 1,
          alpha = 0.1,
          l1_ratio=0.5)

nmf.fit(tfidf)

# Imprime os (number_topics) tópicos com as (number_words) palavras
print_top_words(nmf, dic, number_words)



# NMF 2
print("Rodando NMF (generalized Kullback-Leibler divergence) com tf-idf... \n")

# Salva os nomes das palavras em um dicionário
dic = tfidf_vectorizer.get_feature_names()

nmf = NMF(n_components = number_topics,
          random_state = 1,
          beta_loss = 'kullback-leibler',
          solver = 'mu',
          max_iter=1000,
          alpha=0.1,
          l1_ratio=0.5)

nmf.fit(tfidf)

# Imprime os (number_topics) tópicos com as (number_words) palavras
print_top_words(nmf, dic, number_words)


Rodando NMF (Frobenius norm) com tf-idf... 

Topic 1: {learning - machine - machine learning - performance - methods}

Topic 2: {mining - text - literature - process - study}

Topic 3: {evidence - evidence based - based - study - relevant}

Topic 4: {systematic - classification - method - systematic reviews - review}

Topic 5: {articles - using - classification - relevant - used}

Rodando NMF (generalized Kullback-Leibler divergence) com tf-idf... 

Topic 1: {title - reviews - systematic reviews - systematic - results}

Topic 2: {text - process - techniques - review - support}

Topic 3: {research - evidence - study - used - methods}

Topic 4: {evidence - classification - evidence based - systematic - keywords}

Topic 5: {based - mining - relevant - articles - time}



## TESTES v.1

Iremos testar o algoritmo com a lógica em que o AND fica entre as palavras do mesmo tópico e OR entre os tópicos, realizado sobre a base de dados apenas dos metadados (apenas título, abstract e keywords), uma vez que foi a que retornou os melhores resultados no LDA.

Note que existem 2 execuções para o NMF (Frobenius Norm e Generalized Kullback-Leibler Divergence)

-------------------------------------------------

### 1° Teste: 2 Tópicos e 5 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {learning - machine - articles - systematic - classification}

Topic 2: {mining - systematic - text - process - literature}

String: TITLE-ABS-KEY(("learning" AND "machine" AND "articles" AND "systematic" AND "classification") OR ("mining" AND "systematic" AND "text" AND "process" AND "literature"))

Busca: 317 documentos encontrados 

**Resultado**

Topic 1: {systematic - title - reviews - systematic reviews - keywords}

Topic 2: {text - process - techniques - mining - research}

String: TITLE-ABS-KEY(("systematic" AND "title" AND "reviews" AND "systematic reviews" AND "keywords") OR ("text" AND "process" AND "techniques" AND "mining" AND "research"))

Busca: 1003 documentos encontrados 

-------------------------------------------------

### 2° Teste: 3 Tópicos e 5 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {learning - machine - articles - systematic - machine learning}

Topic 2: {mining - systematic - text - process - method}

Topic 3: {evidence - based - evidence based - study - relevant}

String: TITLE-ABS-KEY(("learning" AND "machine" AND "articles" AND "systematic" AND "machine learning") OR ("mining" AND "systematic" AND "text" AND "process" AND "method") OR ("evidence" AND "based" AND "evidence based" AND "study" AND "relevant"))

Busca: 11,614 documentos encontrados

**Resultado**

Topic 1: {systematic - title - reviews - systematic reviews - keywords}

Topic 2: {text - process - techniques - mining - research}

Topic 3: {study - evidence - evidence based - methods - research}

String: TITLE-ABS-KEY(("studies" AND "study" AND "selection" AND "process" AND "review") OR ("author" AND "data" AND "document" AND "present" AND "systematic") OR ("review" AND "classii" AND "data" AND "set" AND "systematic"))

Busca: 12,741 documentos encontrados

-------------------------------------------------

### 3° Teste: 4 Tópicos e 5 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {learning - machine - articles - machine learning - reviews}

Topic 2: {mining - text - literature - process - study}

Topic 3: {evidence - based - evidence based - study - relevant}

Topic 4: {systematic - method - classification - systematic reviews - systematic review}

String: TITLE-ABS-KEY(("learning" AND "machine" AND "articles" AND "machine learning" AND "reviews") OR ("mining" AND "text" AND "literature" AND "process" AND "study") OR ("evidence" AND "based" AND "evidence based" AND "study" AND "relevant") OR ("systematic" AND "method" AND "classification" AND "systematic reviews" AND "systematic review"))

Busca: 16,460 documentos encontrados

**Resultado**

Topic 1: {title - reviews - systematic - systematic reviews - results}

Topic 2: {text - process - techniques - mining - review}

Topic 3: {research - evidence - study - used - methods}

Topic 4: {evidence - classification - evidence based - keywords - systematic}

String: TITLE-ABS-KEY(("title" AND "reviews" AND "systematic" AND "systematic reviews" AND "results") OR ("text" AND "process" AND "techniques" AND "mining" AND "review") OR ("research" AND "evidence" AND "study" AND "used" AND "methods") OR ("evidence" AND "classification" AND "evidence based" AND "keywords" AND "systematic"))

Busca: 38,152 documentos encontrados

-------------------------------------------------

### 4° Teste: 2 Tópicos e 10 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {learning - machine - articles - systematic - classification - machine learning - based - reviews - systematic reviews - performance}

Topic 2: {mining - systematic - text - process - literature - study - method - review - support - results}

String: TITLE-ABS-KEY(("learning" AND "machine" AND "articles" AND "systematic" AND "classification" AND "machine learning" AND "based" AND "reviews" AND "systematic reviews" AND "performance") OR ("mining" AND "systematic" AND "text" AND "process" AND "literature" AND "study" AND "method" AND "review" AND "support" AND "results"))


Busca: 24 documentos encontrados

**Resultado**

Topic 1: {systematic - title - reviews - systematic reviews - keywords - results - review - used - evidence - based}

Topic 2: {text - process - techniques - mining - research - support - method - results - literature - review}

String: TITLE-ABS-KEY(("systematic" AND "title" AND "reviews" AND "systematic reviews" AND "keywords" AND "results" AND "review" AND "used" AND "evidence" AND "based") OR ("text" AND "process" AND "techniques" AND "mining" AND "research" AND "support" AND "method" AND "results" AND "literature" AND "review"))


Busca: 48 documentos encontrados

-------------------------------------------------

### 5° Teste: 2 Tópicos e 3 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {learning - machine - articles}

Topic 2: {mining - systematic - text}

String: TITLE-ABS-KEY(("learning" AND "machine" AND "articles") OR ("mining" AND "systematic" AND "text"))

Busca: 24,639 documentos encontrados

**Resultado**

Topic 1: {systematic - title - reviews}

Topic 2: {text - process - techniques}

String: TITLE-ABS-KEY(("systematic" AND "title" AND "reviews") OR ("text" AND "process" AND "techniques"))

Busca: 565,819 documentos encontrados

-------------------------------------------------

### 6° Teste: 3 Tópicos e 10 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {author - data - document - present - systematic - set - sensitivity - learning - performance - topics}

Topic 2: {review - classiï - data - set - systematic - learning - reviews - text - articles - based}

Topic 3: {studies - study - selection - process - review - primary - systematic - software - text - mining}

String: TITLE-ABS-KEY(("author" AND "data" AND "document" AND "present" AND "systematic" AND "set" AND "sensitivity" AND "learning" AND "performance" AND "topics") OR ("review" AND "clasifii" AND "data" AND "set" AND "systematic" AND "learning" AND "reviews" AND "text" AND "articles" AND "based") OR ("studies" AND "study" AND "selection" AND "process" AND "review" AND "primary" AND "systematic" AND "software" AND "text" AND "mining"))

Busca: 6 documentos encontrados

-------------------------------------------------

### 7° Teste: 1 Tópicos e 10 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {studies - systematic - review - data - study - text - set - reviews - results - used}

String:  TITLE-ABS-KEY(("studies" AND "systematic" AND "review" AND "data" AND "study" AND "text" AND "set" AND "reviews" AND "results" AND "used"))

Busca: 115 documentos encontrados ...

-------------------------------------------------

### 8° Teste: 5 Tópicos e 2 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {study - process}

Topic 2: {studies - study}

Topic 3: {author - document}

Topic 4: {set - classiï}

Topic 5: {learning - reviews}


String: TITLE-ABS-KEY(("study" AND "process") OR ("studies" AND "study") OR ("author" AND "document") OR ("set" AND "classii") OR ("learning" AND "review"))

Busca: 23,126,784 documentos encontrados

-------------------------------------------------

È notório que os teses resultaram em péssimos resultados, uma vez que em quase todas as buscas o resultado não convergia, explodindo o número de artigos retornados.

# DÚVIDA: 46 DOS 52 ARTIGOS ESTÃO NA BASE SCOPUS, MESMO ASSIM FAZ SENTIDO UTILIZAR OS 52 AO RODAR O LDA?

# olhar este artigo https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4662004/